# Assignment 3, Part 2: T5 SQuAD Model 

Welcome to the part 2 of testing the models for this week's assignment. This time we will perform decoding using the T5 SQuAD model. In this notebook we'll perform Question Answering by providing a "Question", its "Context" and see how well we get the "Target" answer. 

## IMPORTANT

- As you cannot save the changes you make to this colab, you have to make a copy of this notebook in your own drive and run that. You can do so by going to `File -> Save a copy in Drive`. Close this colab and open the copy which you have made in your own drive.

- Go to this [google drive folder](https://drive.google.com/drive/folders/1rOZsbEzcpMRVvgrRULRh1JPFpkIG_JOz?usp=sharing) named `NLP C4 W3 Data`. In the folder, next to its name use the drop down menu to select `"Add shortcut to Drive" -> "My Drive" and then press ADD SHORTCUT`. This should add a shortcut to the folder `NLP C4 W3 Data` within your own google drive. Please make sure this happens, as you'll be reading the data for this notebook from this folder.

- Make sure your runtime is GPU (_not_ CPU or TPU). And if it is an option, make sure you are using _Python 3_. You can select these settings by going to `Runtime -> Change runtime type -> Select the above mentioned settings and then press SAVE`

**Note: Restarting the runtime maybe required**.

Colab will tell you if the restarting is necessary -- you can do this from the:

Runtime > Restart Runtime

option in the dropdown.

## Outline

- [Part 0: Downloading and loading dependencies](#0)
- [Part 1: Mounting your drive for data accessibility](#1)
- [Part 2: Getting things ready](#2)
- [Part 3: Fine-tuning on SQuAD](#3)
    - [3.1 Loading in the data and preprocessing](#3.1)
    - [3.2 Decoding from a fine-tuned model](#3.2)

<a name='0'></a>
# Part 0: Downloading and loading dependencies

Uncomment the code cell below and run it to download some dependencies that you will need. You need to download them once every time you open the colab. You can ignore the `kfac` error.

In [1]:
%%capture
!pip install trax

In [6]:
!pip list | grep 'trax\|t5'

t5                            0.8.1                
trax                          1.3.7                


In [4]:
import string
import t5
import numpy as np
import trax 
from trax.supervised import decoding
import textwrap 
# Will come handy later.
wrapper = textwrap.TextWrapper(width=70)

<a name='2'></a>
# Part 2: Getting things ready 

Run the code cell below to ready some functions which will later help us in decoding. The code and the functions are the same as the ones you previsouly ran on the coursera version of the assignment.

In [11]:
PAD, EOS, UNK = 0, 1, 2

t5_data_path = "/content/drive/MyDrive/NLP C4 W3 Data/"

def detokenize(np_array):
    return trax.data.detokenize(
        np_array,
        vocab_type="sentencepiece",
        vocab_file="sentencepiece.model",
        vocab_dir=t5_data_path
    )

def tokenize(s):
  # The trax.data.tokenize function operates on streams,
  # that's why we have to create 1-element stream with iter
  # and later retrieve the result with next.
  return next(trax.data.tokenize(
      iter([s]),
      vocab_type='sentencepiece',
      vocab_file='sentencepiece.model',
      vocab_dir=t5_data_path))
  
vocab_size = trax.data.vocab_size(
    vocab_type='sentencepiece',
    vocab_file='sentencepiece.model',
    vocab_dir=t5_data_path)

def get_sentinels(vocab_size):
    sentinels = {}

    for i, char in enumerate(reversed(string.ascii_letters), 1):

        decoded_text = detokenize([vocab_size - i]) 
        
        # Sentinels, ex: <Z> - <a>
        sentinels[decoded_text] = f'<{char}>'
        
    return sentinels

sentinels = get_sentinels(vocab_size)    


def pretty_decode(encoded_str_list, sentinels=sentinels):
    # If already a string, just do the replacements.
    if isinstance(encoded_str_list, (str, bytes)):
        for token, char in sentinels.items():
            encoded_str_list = encoded_str_list.replace(token, char)
        return encoded_str_list
  
    # We need to decode and then prettyfy it.
    return pretty_decode(detokenize(encoded_str_list))  

<a name='3'></a>
# Part 3: Fine-tuning on SQuAD

Now let's try to fine tune on SQuAD and see what becomes of the model. For this, we need to write a function that will create and process the SQuAD `tf.data.Dataset`. Below is how T5 pre-processes SQuAD dataset as a text2text example. Before we jump in, we will have to first load in the data. 

<a name='3.1'></a>
### 3.1 Loading in the data and preprocessing

You first start by loading in the dataset. The text2text example for a SQuAD example looks like:

```
{
  'inputs': 'question: <question> context: <article>',
  'targets': '<answer_0>',
}
```

The squad pre-processing function takes in the dataset and processes it using the sentencePiece vocabulary you have seen above. It generates the features from the vocab and encodes the string features. It takes on question, context, and answer, and returns "question: Q context: C" as input and "A" as target.

In [12]:
# Retrieve Question, C, A and return "question: Q context: C" as input 
# and "A" as target.
def squad_preprocess_fn(dataset, mode="train"):
    return t5.data.preprocessors.squad(dataset)


In [13]:
# train generator, this takes about 1 minute
train_generator_fn, eval_generator_fn = trax.data.tf_inputs.data_streams(
    "squad/v1.1:2.0.0",
    data_dir=f"{t5_data_path}data/",
    bare_preprocess_fn=squad_preprocess_fn,
    input_name="inputs",
    target_name="targets"
)
train_generator = train_generator_fn()
next(train_generator)

(b'question: When was the USB Battery Charging Specification Revision 1 . 1 released ? context: The USB Battery Charging Specification Revision 1 . 1 ( released in 2007 ) defines a new type of USB port , called the charging port . Contrary to the standard downstream port , for which current draw by a connected portable device can exceed 100 mA only after digital negotiation with the host or hub , a charging port can supply currents between 500 mA and 1 . 5 A without the digital negotiation . A charging port supplies up to 500 mA at 5 V , up to the rated current at 3 . 6 V or more , and drops its output voltage if the portable device attempts to draw more than the rated current . The charger port may shut down if the load is too high . ',
 b'in 2007')